In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR = '../../datasets/NNN/'

all_ks = {}
to_remove = []
for _r in ['face', 'body', 'object']:
    subdir = os.path.join(DATA_DIR, f'{_r}_mins.pkl')
    df = pd.read_pickle(subdir)
    all_ks.update(df)

    SAVE_DIR = '../../datasets/NNN/'
    ed_df = pd.read_pickle(os.path.join(DATA_DIR, f'{_r}_ed.pkl'))
    # find ROIs that have any NaN ED
    bad_rois = (
        ed_df
        .groupby('ROI')['ED']
        .apply(lambda x: x.isna().any())
    )
    
    to_remove.extend(list(bad_rois[bad_rois].index))
    
dat = (
    pd.DataFrame.from_dict(all_ks, orient='index', columns=['k', 'score'])
    .reset_index()
    .rename(columns={'index': 'ROI'})
)

# keep only ROIs with no NaN EDs
dat = dat[~dat['ROI'].isin(to_remove)].reset_index(drop=True)

dat['Group'] = dat['ROI'].str[-1]   # F, B, O
dat['Group'].value_counts()

SAVE_DIR = '../../../buckets/manifold-dynamics/'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(2,2))

sns.boxplot(data=dat, x='Group', y='k',
            fill=False, color='black', ax=ax)
sns.stripplot(data=dat, x='Group', y='k',
            color='red', alpha=0.5, ax=ax)

ax.set_ylabel('k value')
ax.set_xlabel('Major selectivity')
ax.set_xticks(range(3))
ax.set_xticklabels(['Face', 'Body', 'Object'])
# ax.set_title('optimal k per ROI')
ax.tick_params(axis='x', labelrotation=45)

sns.despine(fig=fig, offset=5, trim=True)
out_path = os.path.join(SAVE_DIR, 'optimal_k_values.png')
plt.savefig(out_path, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
DATA_DIR = '../../datasets/NNN/'

# load in all data and consolidate into one big df
df = pd.DataFrame()
CATS = ['face', 'body', 'object']
for CAT in ['face', 'body', 'object']:
    cat_df = pd.read_pickle(os.path.join(DATA_DIR, f'{CAT}_ed.pkl'))
    no_nans = cat_df.groupby('ROI').filter(lambda x: not x.isna().any().any())
    df = pd.concat([df, no_nans])

# extract per-ROI global ED baseline
g = (
    df[df['Method'] == 'global']
    .loc[:, ['ROI', 'ED']]
    .rename(columns={'ED': 'ED_global'})
)
df = df.merge(g, on='ROI', how='left')

# calculate % change (relative to global) and group by ROI (for bootstrap)
df['percent_change'] = (df['ED'] - df['ED_global']) / df['ED_global'] * 100

df_rel = df[df['Method'] != 'global'].copy()
roi_agg = (df_rel.groupby(['ROI', 'Method'])['percent_change'].median().reset_index())

# map last letter → family
family_map = {'F': 'face', 'B': 'body', 'O': 'object'}

roi_agg['family'] = (
    roi_agg['ROI']
    .str[-1]                 # take last character
    .map(family_map)         # map to family
)

In [ ]:
customp = sns.color_palette('Dark2', 3)
customp[2], customp[1] = customp[1], customp[2]

fig,ax = plt.subplots(1,1, figsize=(4,2))

sns.barplot(roi_agg, x='family', y='percent_change', hue='Method', order = CATS,
            errorbar = 'se', err_kws = {'color': 'black', 'lw': 1}, palette = customp, ax=ax)
sns.stripplot(roi_agg, x='family', y='percent_change', hue='Method',order=CATS, 
              dodge=True, alpha=0.5, marker='.', color = 'black', ax=ax)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:len(roi_agg['Method'].unique())],
          ['Top', 'Rand.'],
          loc='lower right')

ax.set_xlabel('Major selectivity')
ax.set_ylabel('% ED change')
ax.set_xticks(range(3))
ax.set_xticklabels(['Face', 'Body', 'Object'])

sns.despine(fig=fig, trim=True, offset=5)
out_path = os.path.join(SAVE_DIR, 'ed_summary.png')
plt.savefig(out_path, dpi=300, bbox_inches='tight', transparent=True)